<a href="https://colab.research.google.com/github/vrmshamba/vrmshamba/blob/main/Chat_bot_for_collecting_data_on_the_amana_market_platfrom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install --upgrade google-generativeai # upgrade google-generativeai
!pip install --upgrade langchain # upgrade langchain
!pip install langgraph python-dotenv
!pip install langchain-community  # Install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.6 MB/s eta 0:00:00


In [3]:
!pip install --upgrade google-generativeai # upgrade google-generativeai
!pip install --upgrade langchain # upgrade langchain
!pip install langgraph python-dotenv
!pip install langchain-community  # Install langchain-community
# Import statements
from langchain.chat_models import ChatGooglePalm # Use ChatGooglePalm instead of ChatGoogleGenerai
from langchain.schema import BaseMessage # Use langchain.schema instead of langchain_core
from langgraph.graph import StateGraph, END
import os
from dotenv import load_dotenv
import sqlite3 # import sqlite3


# Load environment variables
load_dotenv()
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")

# Initialize LLM
llm = ChatGooglePalm(model="gemini-pro") # Use ChatGooglePalm instead of ChatGoogleGenerai


# Initialize SQLite database (for demonstration purposes, using in-memory)
conn = sqlite3.connect(":memory:")  # Or "amana_market.db" for a file
cursor = conn.cursor()
cursor.execute("""
CREATE TABLE IF NOT EXISTS prices (
    product TEXT,
    price TEXT,
    availability TEXT,
    location TEXT,
    date TEXT
)
""")
conn.commit()


# Define the nodes of the graph
def collect_product_name(state):
    messages = state.get("messages", [])  # Get existing messages or default to []
    messages.append(("human", "What is the name of the product?"))
    response = llm.invoke(messages)
    return {"messages": [*messages, response], "product": response.content}


def collect_price(state):
    messages = state["messages"]
    messages.append(("human", "What is the price and unit of measurement?"))
    response = llm.invoke(messages)
    return {"messages": [*messages, response], "price": response.content, "product": state["product"]}


def collect_availability(state):
    messages = state["messages"]
    messages.append(("human", "Is the product currently available? (In Stock, Out of Stock, Seasonal)"))
    response = llm.invoke(messages)
    return {"messages": [*messages, response], "availability": response.content, "product": state["product"],
            "price": state["price"]}


def collect_location(state):
    messages = state["messages"]
    messages.append(("human", "Where in the market is the product located?"))
    response = llm.invoke(messages)
    return {"messages": [*messages, response], "location": response.content, "product": state["product"],
            "price": state["price"], "availability": state["availability"]}



def store_data(state):
    product = state["product"]
    price = state["price"]
    availability = state["availability"]
    location = state["location"]
    cursor.execute("INSERT INTO prices VALUES (?, ?, ?, ?, DATE('now'))",
                   (product, price, availability, location))
    conn.commit()
    return {"messages": [("ai", "Data stored successfully!")]}



# Define the graph
workflow = StateGraph(BaseMessage)
workflow.add_node("collect_product_name", collect_product_name)
workflow.add_node("collect_price", collect_price)
workflow.add_node("collect_availability", collect_availability)
workflow.add_node("collect_location", collect_location)
workflow.add_node("store_data", store_data)

workflow.set_entry_point("collect_product_name")
workflow.add_edge("collect_product_name", "collect_price")
workflow.add_edge("collect_price", "collect_availability")
workflow.add_edge("collect_availability", "collect_location")
workflow.add_edge("collect_location", "store_data")
workflow.add_edge("store_data", END)  # End the graph after storing data

chain = workflow.compile()

# Run the graph
inputs = {}  # Start with empty input for the first node
for output in chain.stream(inputs):
    for key, value in output.items():
        if key == "messages":
            for message in value:
                print(f"{message[0]}: {message[1]}")
    print("\n---")
conn.close()

TypeError: str expected, not NoneType